In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import os
import sys

In [2]:
dir_current = os.path.abspath('')
dir_parent  = os.path.dirname(dir_current)
if not dir_parent in sys.path: sys.path.append(dir_parent)

In [3]:
tf.random.set_seed(1234)
np.random.seed(1234)

## Loading Custom Dataset

In [ ]:
# Downloading The Dataset
ds = tfds.load('deep_weeds', split='train', shuffle_files=True)
assert isinstance(ds, tf.data.Dataset)
print(ds)

In [4]:
x_train, y_train = tfds.as_numpy(tfds.load('deep_weeds', split='train[:80%]', batch_size=-1, as_supervised=True))
x_valid, y_valid = tfds.as_numpy(tfds.load('deep_weeds', split='train[80:85%]', batch_size=-1, as_supervised=True))
x_test,  y_test  = tfds.as_numpy(tfds.load('deep_weeds', split='train[85%:]', batch_size=-1, as_supervised=True))

## Creating DataSet Objects

In [1]:
Nclasses = 9
def to_categorical(x_, y_):
    return x_, tf.one_hot(y_, depth = Nclasses)
def normalize(x_, y_):
    return tf.cast(x_, tf.float32) / 255., y_

In [8]:
from functions import AlexNet_resize

bs = 32

# Training
#-----------------------------------------
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
# Shuffle
train_dataset = train_dataset.shuffle(buffer_size=x_train.shape[0])
train_dataset = train_dataset.map(to_categorical)
train_dataset = train_dataset.map(normalize)
train_dataset = train_dataset.map(AlexNet_resize)
# Divide in batches
train_dataset = train_dataset.batch(bs)
# Repeat
train_dataset = train_dataset.repeat()

#Validation   
# -----------------------
valid_dataset = tf.data.Dataset.from_tensor_slices((x_valid, y_valid))
# Enconding
valid_dataset = valid_dataset.map(to_categorical)
# Normalize
valid_dataset = valid_dataset.map(normalize)
# Resize
valid_dataset = valid_dataset.map(AlexNet_resize)
# Divide in batches
valid_dataset = valid_dataset.batch(bs)
# Repeat
valid_dataset = valid_dataset.repeat()

#Testing 
# -------------------
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = test_dataset.map(to_categorical)
test_dataset = test_dataset.map(normalize)
test_dataset = test_dataset.map(AlexNet_resize)
test_dataset = test_dataset.batch(1)

## Building Model

In [4]:
from models import AlexNet_body

input_layer   = tf.keras.Input((227,227,3))
output_layer  = AlexNet_body(input_layer, Quantization = False, N_labels = Nclasses)

AlexNet  = tf.keras.Model(inputs=input_layer, outputs=output_layer)
AlexNet.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 227, 227, 3)]     0         
_________________________________________________________________
lambda (Lambda)              (None, 227, 227, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 55, 55, 96)        34944     
_________________________________________________________________
lambda_1 (Lambda)            (None, 55, 55, 96)        0         
_________________________________________________________________
tf_op_layer_Relu (TensorFlow [(None, 55, 55, 96)]      0         
_________________________________________________________________
lambda_2 (Lambda)            (None, 55, 55, 96)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 55, 55, 96)        384   

## Training Options

In [22]:
# Optimization params
# -------------------

# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Validation metrics
# ------------------

metrics = ['accuracy']
# ------------------

# Compile Model
AlexNet.compile(optimizer=optimizer, loss=loss, metrics=metrics)

## Callbacks

In [24]:
early_stop = True
cwd = os.getcwd()
callbacks = []

# Early Stopping
# --------------
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    callbacks.append(es_callback)


## Training

In [25]:
AlexNet.fit(x=train_dataset,
          epochs=100,  #### set repeat in training dataset
          steps_per_epoch=int(np.ceil(x_train.shape[0] / bs)),
          validation_data=valid_dataset,
          validation_steps=int(np.ceil(x_valid.shape[0] / bs)), 
          callbacks=callbacks)

Train for 33 steps, validate for 1 steps
Epoch 1/100
33/33 [==============================] - 6s 190ms/step - loss: 1.0431 - accuracy: 0.8296 - val_loss: 0.9464 - val_accuracy: 0.3846
Epoch 2/100
33/33 [==============================] - 1s 45ms/step - loss: 0.2345 - accuracy: 0.9581 - val_loss: 1.7026 - val_accuracy: 0.3846
Epoch 3/100
33/33 [==============================] - 2s 49ms/step - loss: 0.0639 - accuracy: 0.9844 - val_loss: 2.4316 - val_accuracy: 0.3846
Epoch 4/100
33/33 [==============================] - 2s 56ms/step - loss: 0.0921 - accuracy: 0.9796 - val_loss: 4.0822 - val_accuracy: 0.3846
Epoch 5/100
33/33 [==============================] - 2s 56ms/step - loss: 0.1180 - accuracy: 0.9796 - val_loss: 7.4880 - val_accuracy: 0.3846
Epoch 6/100
33/33 [==============================] - 2s 57ms/step - loss: 0.1092 - accuracy: 0.9805 - val_loss: 5.3382 - val_accuracy: 0.3846


## Saving Weights

In [9]:
Wgt_dir = os.path.join(cwd, 'TrainedWeights')
Wgt_dir = os.path.join(Wgt_dir, 'WeedWeights')
Wgt_dir = os.path.join(Wgt_dir,'Weights')
AlexNet.save_weights(Wgt_dir)